In [57]:
import os
import glob
import numpy as np

import torch

import warnings
from src.BiomassImprovedCNN import BiomassImprovedCNN
from src.BiomassTransformer import BiomassTransformer
from src.BiomassDINOv3 import BiomassDINOv3

from src.DINOv3Wrapper import DINOv3InferenceWrapper
from src.TransformerWrapper import TransformerInferenceWrapper
from src.CNNWrapper import InferenceWrapper

warnings.simplefilter(action='ignore', category=FutureWarning)
print(f"PyTorch: {torch.__version__}")
print(f"Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

PyTorch: 2.9.1+cu128
Device: NVIDIA GeForce RTX 5050 Laptop GPU


In [58]:
CKPT_FOLDER = "./kaggle/checkpoints/improved_cnn/fold1/local_facebook/"

In [59]:
# Competition metric definition (required for checkpoint loading)
labels = ["Dry_Clover_g", "Dry_Dead_g", "Dry_Green_g", "Dry_Total_g", "GDM_g"]

weights = {
    'Dry_Clover_g': 0.1,
    'Dry_Dead_g': 0.1,
    'Dry_Green_g': 0.1,
    'Dry_Total_g': 0.5,
    'GDM_g': 0.2,
}

def competition_metric(y_true, y_pred) -> float:
    """Calculate competition's weighted R2 score."""
    weights_array = np.array([weights[l] for l in labels])

    y_weighted_mean = np.average(y_true, weights=weights_array, axis=1).mean()

    ss_res = np.average((y_true - y_pred)**2,
                        weights=weights_array, axis=1).mean()
    ss_tot = np.average((y_true - y_weighted_mean)**2,
                        weights=weights_array, axis=1).mean()

    return 1 - ss_res / ss_tot

In [60]:
def get_comp_metric(ckpt_path: str) -> float:
    """Extract competition metric from checkpoint filename."""
    return float(ckpt_path.split('val_r2_score=')[-1].split('.ckpt')[0])

In [61]:
def choose_best_checkpoint(ckpt_folder: str) -> str:
    """Choose the best checkpoint based on validation RMSE in filename."""
    ckpt_files = glob.glob(os.path.join(ckpt_folder, "*.ckpt"))
    best_r2 = -np.inf

    for ckpt in ckpt_files:
        r2 = get_comp_metric(ckpt)
        if r2 > best_r2:
            best_r2 = r2
            best_ckpt = ckpt

    for i, ckpt in enumerate(ckpt_files):
        print(f"[{i}] {ckpt} --> R2: {get_comp_metric(ckpt):.6f}")

    print(f"Best checkpoint: {best_ckpt} with R2: {best_r2:.6f}")
    best_ckpt_id = input("Choose the index of the best checkpoint to use ('Enter' for default): ")
    if best_ckpt_id != '':
        best_ckpt = ckpt_files[int(best_ckpt_id)]
        print(f"Selected checkpoint: {best_ckpt}")
    print(f"Using checkpoint: {best_ckpt}")
    return best_ckpt

In [62]:
def get_output_path(ckpt_path: str) -> str:
    """Generate output filename based on checkpoint name."""
    base_name = os.path.basename(ckpt_path).replace('.ckpt', '.pt')
    return os.path.join(os.path.dirname(ckpt_path), base_name)

In [63]:
def get_model_class(ckpt_path: str) -> type:
    """Return appropriate model wrapper based on checkpoint filename."""
    ckpt_path = ckpt_path.lower()
    print(ckpt_path)
    
    if 'dinov3' in ckpt_path:
        return BiomassDINOv3
    elif 'patch' in ckpt_path or 'vit' in ckpt_path:
        return BiomassTransformer
    
    return BiomassImprovedCNN

In [64]:
def get_size_mean_std(ckpt_path: str) -> tuple[int, list[float], list[float]]:
    """Get input size, mean, and std for a given model."""
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    img_size = 224  # Default

    ckpt_path = ckpt_path.lower()
    
    if 'convnextv2_tiny' in ckpt_path:
        img_size = 384
        print(f"✅ Config hardcoded for ConvNeXtV2 Tiny")
    elif any(x in ckpt_path for x in ['vit_large_patch14_dinov2.lvd142m', 'vit_giant_patch14_dinov2.lvd142m']):
        img_size = 518
        print(f"✅ Config hardcoded for DINOv2 ViT Large/Giant")

    return img_size, mean, std

In [69]:
def get_model_wrapper(ckpt_path: str, model: type) -> tuple[InferenceWrapper | TransformerInferenceWrapper | DINOv3InferenceWrapper, int]:
    """Return appropriate model wrapper based on model type."""
    # Choose appropriate wrapper based on model type
    is_dinov3 = isinstance(model, BiomassDINOv3)
    is_transformer = isinstance(model, BiomassTransformer)

    input_size, mean, std = get_size_mean_std(ckpt_path)

    wrapper_kwargs = {
        'lightning_model': model,
        'img_size': input_size,
        'mean': mean,
        'std': std,
    }

    if is_dinov3:
        print("Using DINOv3InferenceWrapper for export...")
        return DINOv3InferenceWrapper(**wrapper_kwargs), input_size
    elif is_transformer:
        print("Using TransformerInferenceWrapper for export...")
        return TransformerInferenceWrapper(**wrapper_kwargs), input_size
    
    print("Using InferenceWrapper for export...")
    return InferenceWrapper(**wrapper_kwargs), input_size

In [66]:
def export_to_torchscript(checkpoint_path: str, output_path: str):
    """Export model to TorchScript for inference without class definition."""

    ModelClass = get_model_class(checkpoint_path)
    print(f"Using model class: {ModelClass.__name__}")

    # Load model
    model = ModelClass.load_from_checkpoint(
        checkpoint_path,
        weights_only=False,
        map_location='cpu'
    )
    model.eval()

    # Wrap model for export
    wrapped_model, input_size = get_model_wrapper(checkpoint_path, model)

    print(f"Input size for tracing: {input_size}x{input_size}")

    wrapped_model.eval()

    # Create dummy input
    dummy_left = torch.randn(1, 3, input_size, input_size)
    dummy_right = torch.randn(1, 3, input_size, input_size)

    # Test wrapper first
    print("Testing wrapper before tracing...")
    with torch.no_grad():
        test_output = wrapped_model(dummy_left, dummy_right)
        print(f"Wrapper output shape: {test_output.shape}")
        print(f"Sample prediction: {test_output[0]}")

    # Trace model
    print("\nTracing model...")
    with torch.no_grad():
        traced_model = torch.jit.trace(
            wrapped_model,
            (dummy_left, dummy_right)
        )

    # Save
    traced_model.save(output_path)
    print(f"\nModel exported to: {output_path}")

    # Validate export
    print("\nValidating export...")
    with torch.no_grad():
        original_output = wrapped_model(dummy_left, dummy_right)
        traced_output = traced_model(dummy_left, dummy_right)
        max_diff = (original_output - traced_output).abs().max().item()
        print(f"Max difference between original and traced: {max_diff:.8f}")

        if max_diff < 1e-5:
            print("✅ Export successful!")
        else:
            print(f"⚠️ Export may have issues (difference: {max_diff})")


In [67]:
input_path = choose_best_checkpoint(CKPT_FOLDER)

[0] ./kaggle/checkpoints/improved_cnn/fold1/local_facebook\dinov3-vitl16-pretrain-lvd1689m_train[5]Folds_log_fusion-gated_spatial_cross_epochs15_bs4_gradacc4_lr0.0001_wd0.05_dr0.2_hr0.5-fold1-epoch=04-val_r2_score=0.7666.ckpt --> R2: 0.766600
[1] ./kaggle/checkpoints/improved_cnn/fold1/local_facebook\dinov3-vitl16-pretrain-lvd1689m_train[5]Folds_log_fusion-gated_spatial_cross_epochs15_bs4_gradacc4_lr0.0001_wd0.05_dr0.2_hr0.5-fold1-epoch=10-val_r2_score=0.7823.ckpt --> R2: 0.782300
Best checkpoint: ./kaggle/checkpoints/improved_cnn/fold1/local_facebook\dinov3-vitl16-pretrain-lvd1689m_train[5]Folds_log_fusion-gated_spatial_cross_epochs15_bs4_gradacc4_lr0.0001_wd0.05_dr0.2_hr0.5-fold1-epoch=10-val_r2_score=0.7823.ckpt with R2: 0.782300
Using checkpoint: ./kaggle/checkpoints/improved_cnn/fold1/local_facebook\dinov3-vitl16-pretrain-lvd1689m_train[5]Folds_log_fusion-gated_spatial_cross_epochs15_bs4_gradacc4_lr0.0001_wd0.05_dr0.2_hr0.5-fold1-epoch=10-val_r2_score=0.7823.ckpt


In [70]:
export_to_torchscript(
    checkpoint_path=input_path,
    output_path=get_output_path(input_path)
)

./kaggle/checkpoints/improved_cnn/fold1/local_facebook\dinov3-vitl16-pretrain-lvd1689m_train[5]folds_log_fusion-gated_spatial_cross_epochs15_bs4_gradacc4_lr0.0001_wd0.05_dr0.2_hr0.5-fold1-epoch=10-val_r2_score=0.7823.ckpt
Using model class: BiomassDINOv3
Backbone output dimension: 1024
Using DINOv3InferenceWrapper for export...
Input size for tracing: 224x224
Testing wrapper before tracing...


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Wrapper output shape: torch.Size([1, 3])
Sample prediction: tensor([ 4.5393, 11.6273,  8.0996])

Tracing model...


c:\_GitHub\CSIRO-Image2Biomass-Prediction\.venv\Lib\site-packages\transformers\integrations\sdpa_attention.py:81: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  is_causal = query.shape[2] > 1 and attention_mask is None and getattr(module, "is_causal", True)



Model exported to: ./kaggle/checkpoints/improved_cnn/fold1/local_facebook\dinov3-vitl16-pretrain-lvd1689m_train[5]Folds_log_fusion-gated_spatial_cross_epochs15_bs4_gradacc4_lr0.0001_wd0.05_dr0.2_hr0.5-fold1-epoch=10-val_r2_score=0.7823.pt

Validating export...
Max difference between original and traced: 0.00000000
✅ Export successful!
